# IEEE Fraud detection adversarial validation [sklearn, lgbm]
IEEE Fraud detection train/test data binary classification task.
Reference: <https://www.kaggle.com/code/jtrotman/ieee-fraud-adversarial-lgb-split-points/notebook>

## Import libraries

In [1]:
import os

import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from pandas.api.types import union_categoricals
from sklearn.model_selection import train_test_split

import giskard
from giskard import GiskardClient
from giskard import wrap_dataset, wrap_model

## Define constants

In [2]:
# Constants.
TARGET_COLUMN = 'isTest'
IDX_LABEL = 'TransactionID'

# Paths.
DATA_DIRECTORY = os.path.join(".", "datasets", "fraud_detection")

# Giskard platform credentials.
GISKARD_URL = "http://localhost:19000"
GISKARD_ACCESS_TOKEN = ""

## Data loading and preprocessing

In [3]:
# Define data-types of transactions features.
DATA_TYPES_TRANSACTION = {
    'TransactionID': 'int32',
    'isFraud': 'int8',
    'TransactionDT': 'int32',
    'TransactionAmt': 'float32',
    'ProductCD': 'category',
    'card1': 'int16',
    'card2': 'float32',
    'card3': 'float32',
    'card4': 'category',
    'card5': 'float32',
    'card6': 'category',
    'addr1': 'float32',
    'addr2': 'float32',
    'dist1': 'float32',
    'dist2': 'float32',
    'P_emaildomain': 'category',
    'R_emaildomain': 'category',
}

C_COLS = [f'C{i}' for i in range(1, 15)]
D_COLS = [f'D{i}' for i in range(1, 16)]
M_COLS = [f'M{i}' for i in range(1, 10)]
V_COLS = [f'V{i}' for i in range(1, 340)]

DATA_TYPES_TRANSACTION.update((c, 'float32') for c in C_COLS)
DATA_TYPES_TRANSACTION.update((c, 'float32') for c in D_COLS)
DATA_TYPES_TRANSACTION.update((c, 'float32') for c in V_COLS)
DATA_TYPES_TRANSACTION.update((c, 'category') for c in M_COLS)

In [4]:
# Define datatypes of identity features.
DATA_TYPES_ID = {
    'TransactionID': 'int32',
    'DeviceType': 'category',
    'DeviceInfo': 'category',
}

ID_COLS = [f'id_{i:02d}' for i in range(1, 39)]
ID_CATS = [
    'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
    'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38'
]

DATA_TYPES_ID.update(((c, 'float32') for c in ID_COLS))
DATA_TYPES_ID.update(((c, 'category') for c in ID_CATS))

In [5]:
# Define list of all categorical features.
CATEGORICALS = [f_name for (f_name, f_type) in dict(DATA_TYPES_TRANSACTION, **DATA_TYPES_ID).items() if f_type == "category"]

In [6]:
def read_set(_type):
    """Read both transactions and identity data."""
    print(f"Reading transactions data...")
    _df = pd.read_csv(os.path.join(DATA_DIRECTORY, f'{_type}_transaction.csv'),
                      index_col=IDX_LABEL, dtype=DATA_TYPES_TRANSACTION)

    print(f"Reading identity data...")
    _df = _df.join(pd.read_csv(os.path.join(DATA_DIRECTORY, f'{_type}_identity.csv'),
                               index_col=IDX_LABEL, dtype=DATA_TYPES_ID))
    return _df

def read_dataset():
    """Read whole data."""
    # Read train and test data.
    print("Reading train data...")
    train_set = read_set('train')
    print(f"Train data of shape {train_set.shape} is loaded!")

    print(f"Reading test data...")
    test_set = read_set('test')
    print(f"Test data of shape {test_set.shape} is loaded!")

    return train_set, test_set

def preprocess_dataset(train_set, test_set):
    """Unite train and test into common dataframe."""
    # Create a new target column and remove a former one from the train data.
    print("Start data preprocessing...")
    train_set.pop('isFraud')
    train_set['isTest'] = 0
    test_set['isTest'] = 1

    # Preprocess categorical features.
    n_train = train_set.shape[0]
    for c in train_set.columns:
        s = train_set[c]
        if hasattr(s, 'cat'):
            u = union_categoricals([train_set[c], test_set[c]], sort_categories=True)
            train_set[c] = u[:n_train]
            test_set[c] = u[n_train:]

    # Unite train and test data.
    united = pd.concat([train_set, test_set])

    # Add additional features.
    united['TimeInDay'] = united.TransactionDT % 86400
    united['Cents'] = united.TransactionAmt % 1

    # Remove useless columns.
    united.drop("TransactionDT", axis=1, inplace=True)

    print(f"Dataset merged and preprocessed! Resulted shape: {united.shape}")

    return united

In [7]:
united_dataset = preprocess_dataset(*read_dataset())

Reading train data...
Reading transactions data...
Reading identity data...
Train data of shape (590540, 433) is loaded!
Reading test data...
Reading transactions data...
Reading identity data...
Test data of shape (506691, 432) is loaded!
Start data preprocessing...
Dataset merged and preprocessed! Resulted shape: (1097231, 434)


## Train-test split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(united_dataset.drop(TARGET_COLUMN, axis=1), united_dataset[TARGET_COLUMN], test_size=0.25)

## Wrap test dataset

In [9]:
raw_dataset = pd.concat([X_test, y_test], axis=1)
wrapped_dataset = wrap_dataset(
    dataset=raw_dataset,
    name="fraud_detection_adversarial_dataset",
    target=TARGET_COLUMN,
    cat_columns=CATEGORICALS
)

Your 'pandas.DataFrame' dataset is successfully wrapped by Giskard's 'Dataset' wrapper class.


/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/giskard/core/dataset_validation.py:84: UserWarning: Feature 'V1' is declared as 'numeric' but has 2 (<= category_threshold=5) distinct values. Are you sure it is not a 'category' feature?
  warning(
/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/giskard/core/dataset_validation.py:84: UserWarning: Feature 'V10' is declared as 'numeric' but has 5 (<= category_threshold=5) distinct values. Are you sure it is not a 'category' feature?
  warning(
/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/giskard/core/dataset_validation.py:84: UserWarning: Feature 'V12' is declared as 'numeric' but has 5 (<= category_threshold=5) distinct values. Are you sure it is not a 'category' feature?
  warning(
/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/giskard/core/dataset_validation.py:84: UserWarning: Featu

## Prepare estimator

In [10]:
# Define parameters of an estimator.
ESTIMATOR_PARAMS = {
    'num_leaves': 64,
    'objective': 'binary',
    'min_data_in_leaf': 10,
    'learning_rate': 0.1,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.9,
    'bagging_freq': 1,
    'max_cat_to_onehot': 128,
    'metric': 'auc',
    'n_jobs': -1,
    'seed': 42,
    'subsample_for_bin': united_dataset.shape[0]
}

In [11]:
estimator = LGBMClassifier(**ESTIMATOR_PARAMS)
estimator.fit(X_train, y_train)

[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


LGBMClassifier(bagging_fraction=0.9, bagging_freq=1, feature_fraction=0.5,
               max_cat_to_onehot=128, metric='auc', min_data_in_leaf=10,
               num_leaves=64, objective='binary', seed=42,
               subsample_for_bin=1097231)

In [12]:
train_metric = roc_auc_score(y_train, estimator.predict_proba(X_train)[:, 1].T)
test_metric = roc_auc_score(y_test, estimator.predict_proba(X_test)[:, 1].T)

print(f"Train ROC-AUC score: {train_metric}")
print(f"Test ROC-AUC score: {test_metric}")

Train ROC-AUC score: 0.9197083113299749
Test ROC-AUC score: 0.9178865938658143


## Wrap estimator

In [13]:
wrapped_model = wrap_model(
    model=estimator,
    model_type="classification",
    name="train_test_data_classifier",
    feature_names=X_train.columns,
    classification_threshold=0.5,
    classification_labels=[0, 1]
)

Your 'sklearn' model is successfully wrapped by Giskard's 'SKLearnModel' wrapper class.


## Scan model

In [14]:
scanning_results = giskard.scan(wrapped_model, wrapped_dataset)

/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."
Your model is successfully validated.


In [15]:
display(scanning_results)

/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/giskard/core/dataset_validation.py:84: UserWarning: Feature 'card3' is declared as 'numeric' but has 2 (<= category_threshold=2) distinct values. Are you sure it is not a 'category' feature?
  warning(
/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/giskard/core/dataset_validation.py:84: UserWarning: Feature 'addr2' is declared as 'numeric' but has 1 (<= category_threshold=2) distinct values. Are you sure it is not a 'category' feature?
  warning(
/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/giskard/core/dataset_validation.py:84: UserWarning: Feature 'dist2' is declared as 'numeric' but has 0 (<= category_threshold=2) distinct values. Are you sure it is not a 'category' feature?
  warning(
/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/giskard/core/dataset_validation.py:84: UserWarning

<iframe srcdoc="
<script src="https://cdn.tailwindcss.com"></script>
<script>
 tailwind.config = {
 darkMode: 'class'
 }
</script>
<style>
 table.dataframe {
 width: 100%;
 max-width: 100%;
 overflow: auto;
 }

 .dataframe tr {
 border-bottom: 1px solid #555;
 }

 .dataframe td,
 .dataframe th {
 padding: 0.5rem;
 }
</style>



<div class="dark">
 <div id="gsk-scan" class="dark:text-white dark:bg-zinc-900">
 <!-- TAB HEADER -->
<div class="flex items-center items-stretch h-11">
 <div class="flex items-center px-4 dark:fill-white border-b border-gray-500 h-11">
 <svg xmlns="http://www.w3.org/2000/svg" xml:space="preserve" viewBox="0 0 410 213" height="15">
 <path
 d="M318.78 21.94a59.439 59.439 0 0 1 36.45-12.56v.03c7.09-.1 14.13 1.2 20.71 3.84 6.58 2.64 12.57 6.56 17.62 11.54s9.06 10.91 11.8 17.45a53.44 53.44 0 0 1 0 41.3 53.588 53.588 0 0 1-11.8 17.45c-5.05 4.98-11.04 8.9-17.62 11.54s-13.62 3.95-20.71 3.84l-67.92.16-35.87 49.11-135.24 46.84 40.81-47.24a139.229 139.229 0 0 1-66.06-23.8H45.11L0 101.46h55.73C60.38 44.84 115.97 0 183.59 0c31.71 0 63.58 9.86 87.47 27.04a105.149 105.149 0 0 1 26.65 27.18 59.4 59.4 0 0 1 21.07-32.28zM99.85 46.03l55.4 55.45h21.31l-64.81-64.87c-4.18 2.86-8.15 6.01-11.9 9.42zm57.78-28.45 83.9 83.9h21.29l-86.21-86.22c-6.37.32-12.71 1.1-18.98 2.32zm-92.84 98.94H39.72l11.1 9.82h22.44c-3-3.12-5.83-6.4-8.47-9.82zm134.27 0H84.8l-.03.01c20.8 20.91 51.79 33.67 84.66 34.3l29.63-34.31zm-34.81 63.35 77.79-26.94 26.6-36.39h-49.68l-54.71 63.33zm146.77-78.65h46.2l.05.01c9.98-.55 19.36-4.97 26.14-12.32a38.409 38.409 0 0 0 10.16-27.05 38.44 38.44 0 0 0-11.56-26.48 38.405 38.405 0 0 0-26.74-10.94A44.287 44.287 0 0 0 324 37.41a44.268 44.268 0 0 0-12.97 31.28v32.53zm51.82-52.83c5.27.52 9.12 5.22 8.6 10.49-.52 5.27-5.22 9.12-10.49 8.6s-9.12-5.22-8.6-10.49c.53-5.27 5.22-9.12 10.49-8.6z"
 style="fill-rule:evenodd;clip-rule:evenodd" />
 </svg>
 <div class="ml-4 py-2">
 <span class="uppercase text-sm">15 issues detected</span>
 </div>
 </div>

 
 <div data-tab-target="Model_bias" class="h-12 group active">
 <div
 class="group-[.active]:bg-zinc-800 group-[.active]:border-b-zinc-800 group-[.active]:border-gray-500 border-l px-3 py-2 border-r border-t border-b border-gray-600 border-b-gray-500 h-11">
 Model bias
 
 
 <span class="ml-1 rounded-full text-xs min-w-4 min-h-4 px-1 py-0.5 inline-block text-center bg-red-400">
 
 15
 </span>
 
 </div>
 </div>
 

 <div class="flex-grow border-b border-gray-500 h-11"></div>
</div>
<!-- TAB HEADER END -->
 
 <div id="Model_bias" role="tabpanel" class="dark:text-white dark:bg-zinc-800 p-4 pt-4 mb-4">
 
 <div class="p-3 mt-2 bg-amber-100/40 rounded-sm w-full flex align-middle">
 <div class="text-amber-100">
 <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5"
 stroke="currentColor" class="mt-0.5 w-6 h-6">
 <path stroke-linecap="round" stroke-linejoin="round"
 d="M12 9v3.75m-9.303 3.376c-.866 1.5.217 3.374 1.948 3.374h14.71c1.73 0 2.813-1.874 1.948-3.374L13.949 3.378c-.866-1.5-3.032-1.5-3.898 0L2.697 16.126zM12 15.75h.007v.008H12v-.008z" />
 </svg>
 </div>
 <p class="ml-2 my-1 text-amber-100 text-sm">
 We detected some spots in your data where the model has a tendency to make incorrect
 predictions.
 </p>
 </div>
 

 
 <div class="flex items-center space-x-1">
 <h2 class="uppercase my-4 mr-2 font-medium">Issues</h2>
 
 <span class="text-xs border rounded px-1 uppercase text-red-400 border-red-400">1 major</span>
 
 
 <span class="text-xs border rounded px-1 uppercase text-amber-200 border-amber-200">14 medium</span>
 
 </div>

 <!-- ISSUES TABLE -->
<div class="mb-4">
 <table class="table-auto w-full text-white">
 
 <tbody class="first:border-t border-b border-zinc-400">
 <tr class="gsk-issue text-sm group peer text-left cursor-pointer hover:bg-zinc-700">
 <td class="p-3">
 <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5"
 stroke="currentColor" class="w-4 h-4 group-[.open]:rotate-90">
 <path stroke-linecap="roun

# Upload model and dataset to the Giskard platform

In [16]:
# Define project key.
project_key = "fraud_detection_adversarial_validation"

# Init new giskard client.
client = GiskardClient(GISKARD_URL, GISKARD_ACCESS_TOKEN)

# Create new project.
# project = client.create_project(project_key, "FRAUD_DETECTION_ADVERSARIAL_VALIDATION", "IEEE Fraud detection train/test data classification task.")

# Or get the one already created.
project = client.get_project(project_key)

# Upload the model and the dataset.
model_id = wrapped_model.upload(client, project_key)
dataset_id = wrapped_dataset.upload(client, project_key)

Your model is successfully validated.
Model successfully uploaded to project key 'fraud_detection_adversarial_validation' with ID = 66a16c68-f997-47c6-92ae-52375e5ff627
Dataset successfully uploaded to project key 'fraud_detection_adversarial_validation' with ID = 3dcb65b9-ed10-4b79-8a0d-86ec4dfdd6fb
